# Gait Analysis Activity Classification  

*Application of Data Science Methodology toward classifying the gait activity of subjects via an Inertial Measurement Unit (IMU) motion sensor placed on the top of the foot*  

---

## Introduction

I plan to follow the steps of a data science methodology to address a problem that I had originally worked on while performing research with a kinesiologist. The goal was to be able to classify the activity of the wearer using the sensor measurements taken from an accelerometer/gyrometer placed on the top of the foot.

[Click here to skip ahead and play](#playtime) if you don't want a detailed walkthrough of the methodology :)

## 1. Question or Goal
---

Using 3D accelerometer and gyroscope data gathered from an Inertial Measurement Unit (IMU) placed on the top of the foot of a subject, can we accurately classify their activity (e.g. walking, running, standing, sitting, laying down, etc.)? Further, could refinements be made to integrate the data to model the gait of the subject and analyze their stride length, velocity or other factors?

We can see this potentially as a business problem, perhaps with the goal of developing cheaper diagnostic tools. Or perhaps in applications that can track your workout activities and return relevant statistics to the user.

There are different considerations to be made, but this is intended to serve as a tutorial and demonstration, so we will have to make some hopefully healthy compromises along the way.

## 2. Analytic Approach
---
We have to understand the types of answers we want from this question, and to assess how the data can be used to give those answers.

We need to discuss how the measurements were taken to come up with the correct goals and approaches and models to suss out reasonably accurate and efficient results.

---

### Procedure

- The IMU after being charged, is secured with tape on top of foot.


-  begins recording after being activated by button press while on the foot


- subject is instructed to perform a prescribed set of activities (similar to):
  - stand
  - stomp (3 times)
  - bend foot
  - stomp 
  - walk
  - rest
  - run
  - rest
  - random stuff - for test data walking


- The data is written to `.csv` file that is downloaded directly from the device after a period of recording.

### Measurements

- It measures **acceleration** in g's, a unit of acceleration that is equivalent to the acceleration of gravity (9.81 $m/s^2$).


- It measures **rotational velocity** in degrees per second (deg/s).


- This makes for a total of 6 physical degrees of freedom, and 7 with time. 


- However, the IMU gives us additional data that is not necessary for our analysis, including temperature and 3D magnetometer data. This will be important to clean at the data preparation stage.


## Considerations / Precautions
---

> It's important to scrutinize and consider the errors or constraints inherent in y/our data. One of our goals is to mitigate these errors. 
>
>While I'd rather skip these parts and shove data into a model, it's important to address these as best we can. Sometimes we cannot, and other times we choose not. But it's important to try to acknowledge these aspects of the data


* The IMU is not exactly state-of-the-art, and gyroscope data is very prone to **drift**. We will use **signal processing** techniques like **FFT analysis** to attempt to remove noise and drift.


* We can use *spectral information* from the signals to design different **filters** to see what provides the clearest and most distinct features.


* Incorrect placement of the IMU.
    
> Naturally, it is not going to sit flat on the top of the foot. We must reorient the device and transform the coordinate system to compensate at times, depending on how we want to model things. There is a question about the efficacy and the effort involved in automating this reorientation since it's also possible to classify the activity well enough with an off-center coordinate system. We might however miss out on the potential to perform some real time modeling of the foot/device in the future.
  
I am choosing to forego performing 3D scaling and transformations on the data for the time being and will focus on the signal analysis and model training aspects of the analysis.


# Types of Answers / Goal
---

1. We want to **classify** the activity of the subject (or user).
     * This means we are going to prefer a supervised learning model, a probabilistic model that tells us the most likely outcome or classification from a set of predetermined (supervised) possibilities.
     * We can choose a model and train it to predict the most likely outcomes based off our input training data and the training labels (the correct classifications).  


2. We also want to integrate the accelerometer data to measure average (or real-time) velocity and perhaps integrate that to measure position. We will leverage the information in the lab notes to help in developing accurate models. Another factor is stride length, periodicity (is the left stride longer than the right?).  


3. We may also eventually want to visually model the activity of the foot.  

> ### Okay, that's asking a lot.

That's fine! Let's just make headway toward classifying between three activities: running, walking and standing.

Now we are entering the data requirements phase. We're getting closer to playing with the data, so hang in there.

## 3. Data Requirements
---

* We need to remove the unnecessary data from the dataframe.


* We need to manually label the data with help from `bqplot` and classify it as walking, standing or running so we can train predictive/classification model.


* We need to reorient the data to a body-centered coordinate axis.
    * Or at the very least flatten the coordinate system so that gravity points, (0,0,1) in (x,y,z) acc vector
    * Then we can scale/standardize according to FFT analysis and for extracting features


* We need to filter noise and drift out of the data. (i.e. smooth)

## 4. Data Collection
---

This phase is essentially complete, unless at some point we'd like to get more data to classify different activities. The data only provide enough information for walking and standing and running, and parameters involving those states.

There are physical notes that were taken as well as video, which I don't have access to.


<div id="playtime"><div/>


## 5. Data Understanding / Visualization

Understanding the nature of the data, and the heuristic properties of the experiment and procedure itself will help us to engineer features to look for that may help us classify the activities. Examples might be the maximum acceleration in the x direction, minimum acceleration in the z direction, etc.

Alternatively, we can just visualize the accelerometer data in the X,Y,Z components and see if we notice any patterns. Similarly for rotation.


Let's play around.



In [22]:
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore")
file_path = "./data/TAS1F06180329 (2018-10-24)-IMU.csv"

# The header value has to do with how the CSV is formatted.
# The 10th row contains the column names

IMU_data = pd.read_csv(file_path, header=10)
print("File Read")

File Read


Check that the data look right

In [23]:
IMU_data.head()

,Timestamp,Accelerometer X,Accelerometer Y,Accelerometer Z,Temperature,Gyroscope X,Gyroscope Y,Gyroscope Z,Magnetometer X,Magnetometer Y,Magnetometer Z
0,2018-10-24T11:20:00.0000000,0.046875,-0.008301,1.015625,31.689790,-1.403809,1.403809,-0.549316,-11.132812,7.03125,24.316405
1,2018-10-24T11:20:00.0100000,0.049316,-0.003906,1.016602,31.698775,-1.647949,1.342774,-0.549316,-11.132812,7.03125,24.316405
2,2018-10-24T11:20:00.0200000,0.045898,-0.010742,1.010254,31.695780,-2.258301,1.159668,-0.549316,-11.132812,7.03125,24.316405
3,2018-10-24T11:20:00.0300000,0.044922,-0.010254,1.014160,31.698775,-2.258301,1.098633,-0.549316,-11.132812,7.03125,24.316405
4,2018-10-24T11:20:00.0400000,0.050781,-0.011719,1.008301,31.677809,-2.380371,1.037598,-0.549316,-11.132812,7.03125,24.316405


## Some quick data cleaning

Look at the variables/columns provided by the IMU. We will want to *standardize the column names*--we will do this by casting to lowercase and removing spaces. You'll also notice there's a few more variables other than time, acceleration and gyroscope data. We will `drop` these columns using `df.drop(col_name)`.

In [24]:
IMU_data.columns # or try IMU_data.keys()

Index(['Timestamp', 'Accelerometer X', 'Accelerometer Y', 'Accelerometer Z',
       'Temperature', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z',
       'Magnetometer X', 'Magnetometer Y', 'Magnetometer Z'],
      dtype='object')

In [25]:
# This is how I'm choosing to rename/standardize the columns without the rename function.
new_column_names = []

for val in IMU_data.columns:
    # print(f"before: {val}")
    val = val.lower()
    val = val.replace(" ", "_")
    new_column_names.append(val)
    # print(f"after: {val}")`
    
# Assign new values to old columns
IMU_data.columns = new_column_names

In [26]:
IMU_data.columns.values # can also just use '.columns' without looking at values

array(['timestamp', 'accelerometer_x', 'accelerometer_y',
       'accelerometer_z', 'temperature', 'gyroscope_x', 'gyroscope_y',
       'gyroscope_z', 'magnetometer_x', 'magnetometer_y',
       'magnetometer_z'], dtype=object)


Now I want to `df.drop()` all the undesirable columns: the magnetometer and the temperature data.

In [27]:
undesirables = ["temperature", "magnetometer_x", "magnetometer_y", "magnetometer_z"]

IMU_data.drop(undesirables, # List of col names to drop
              axis=1,       # Axis = 1 specifies columns. Axis = 0 specifies rows.
              inplace=True) # inplace = True means modify orig, False means return copy

### Reindexing for time, instead of date

If the start time is '2018-10-24T11:20:00.0000000', then we obtain the initial time of 0.0s by subtracting that from every timestamp in the data frame. 

|DATE|--->|DATETIME OBJ|--->|UNIX TS|--->|TIME(s)|
|----|----|------------|----|----|----|------------|
|2018-10-24T11:20:00.0000000|--->|DT obj|--->|1540405200.0|--->|0.0
|2018-10-24T11:20:20.0000000|--->|DT obj|--->|1540405220.0|--->|20.0

In [28]:
import dateutil

# map is basically saying apply function to every member
# the argument is a function object, not its call/return
IMU_data['time'] = IMU_data['timestamp'].map(dateutil.parser.isoparse)
IMU_data['time'] = IMU_data['time'].map(datetime.datetime.timestamp)

# Difference between each current second and the initial (removes large unix number)
IMU_data['time'] = IMU_data['time'] - IMU_data['time'].iloc[0]

print(IMU_data.head())

                     timestamp  accelerometer_x  accelerometer_y  \
0  2018-10-24T11:20:00.0000000         0.046875        -0.008301   
1  2018-10-24T11:20:00.0100000         0.049316        -0.003906   
2  2018-10-24T11:20:00.0200000         0.045898        -0.010742   
3  2018-10-24T11:20:00.0300000         0.044922        -0.010254   
4  2018-10-24T11:20:00.0400000         0.050781        -0.011719   

   accelerometer_z  gyroscope_x  gyroscope_y  gyroscope_z  time  
0         1.015625    -1.403809     1.403809    -0.549316  0.00  
1         1.016602    -1.647949     1.342774    -0.549316  0.01  
2         1.010254    -2.258301     1.159668    -0.549316  0.02  
3         1.014160    -2.258301     1.098633    -0.549316  0.03  
4         1.008301    -2.380371     1.037598    -0.549316  0.04  


### Saving the data
Now might be a good time to **save** the groomed csv file to save us trouble/cleaning in the future. We do this using
`df.to_csv()` and specifying a location and file name.

In [29]:
IMU_data.to_csv("./data/Motion_data_stage1.csv",index=False)

Verify this new `.csv` file was saved.

In [30]:
df = pd.read_csv("./data/Motion_data_stage1.csv")
print(df.head(),"\n")


# Or, alternatively, without reading a big file into memory:
#import os
#print(os.path.isfile('Motion_data_stage1.csv'))

                     timestamp  accelerometer_x  accelerometer_y  \
0  2018-10-24T11:20:00.0000000         0.046875        -0.008301   
1  2018-10-24T11:20:00.0100000         0.049316        -0.003906   
2  2018-10-24T11:20:00.0200000         0.045898        -0.010742   
3  2018-10-24T11:20:00.0300000         0.044922        -0.010254   
4  2018-10-24T11:20:00.0400000         0.050781        -0.011719   

   accelerometer_z  gyroscope_x  gyroscope_y  gyroscope_z  time  
0         1.015625    -1.403809     1.403809    -0.549316  0.00  
1         1.016602    -1.647949     1.342774    -0.549316  0.01  
2         1.010254    -2.258301     1.159668    -0.549316  0.02  
3         1.014160    -2.258301     1.098633    -0.549316  0.03  
4         1.008301    -2.380371     1.037598    -0.549316  0.04   



# More Preparation
---
We are still within the **data understanding** stage, and we have done some light data cleaning/preparation.


In order to get accurate real-world measurements, we need to attempt to **correct for noise**, positioning of the foot, and other factors.


>There is a trade off between performing the most accurate cleaning and the most efficient, as it may take too much time/computation to make a model, or even to efficiently and intelligently process signals for input into an ML model--if this cleaning and processing were to be automated and put into production. Overall, for vectorized operations, they will be efficient, but others will be more computationally difficult.

Just a foreshadowing, but we plan on implementing a basic K-Nearest Neighbors model after exploring some features of the data.


### Positioning/acceleration - Heuristic stuff

At rest, the foot has one unit of acceleration applied opposite of the z-axis. In a 3-dimensional, xyz-coordinate, system this would give a measurement of (0g,0g,1g). Let's average the acceleration of the resting foot to determine if this is the case.

> note that you would not simply just subtract (0,0,1) from all inputs. Since the angle of the foot changes, you need to get the angle of rotation of the foot, and then subsequently rotate a vector (0,0,1) in that same direction, and then subtract the resulting vector from the data.
>
>
> > This physical element would be super fun to implement but for now it is a <span style="color: orange">TODO</span>. Again, we will focus on the signal processing aspects of this analysis.

#  Getting Data Ready For BQPLOT

Here we want to define start and end intervals to view the accelerometer data. We can do this manually, but I want to generate some interactivity with a fast and awesome tool created by developers at Bloomberg, `bqplot`. It is a `jupyter` based `d3.js` bridge, and functions as a python library which implements a JavaScript backend, and utilizes `ipywidgets` to create interactive javascript widgets.


It is becoming easier to make your own custom widgets, as well as interactive dashboards, with tools like BQPLOT. These data were difficult to load on matplotlib backend, and its interactivity was not suited for larger data sets.

In [37]:
sec = 100        # 100 samples per second 
num_secs = 8000    # how much we want to view

start = 20 * sec
end = start + (num_secs * sec)

# iloc allows you to slice from and/or locate entries by integer indices.
x = df['time'].iloc[start:end]
y = [df.iloc[start:end]['accelerometer_x'],
    df.iloc[start:end]['accelerometer_y'],
    df.iloc[start:end]['accelerometer_z'],
    df.iloc[start:end]['gyroscope_x'],
    df.iloc[start:end]['gyroscope_y'],
    df.iloc[start:end]['gyroscope_z']]

print(f"Length of data (s):  {IMU_data.size/sec}")
print(f"Length of selected (s): {(end-start)/sec}")

Length of data (s):  9592.0
Length of selected (s): 8000.0


## Load `bqplot` - Define Graphical Objects

Graphical objects are displayed as `bqplot.figures`.

The main ingredients of figures are **Scales**, **Marks** and **Interacts**:

* Scales ~ Axes objects
* Marks ~ Line, Scatter, Hist objects  (See [Documentation](https://bqplot.readthedocs.io/en/latest/api_documentation.html#marks))
* Interacts ~ GUI / How the objects update in real time 😀
    * Sliders, toggles, selectors, etc


In [38]:
import bqplot as bq
from bqplot import pyplot as plt

# Scales are where a lot of bqplot's magic happens
scales = {'x': bq.LinearScale(), 
          'y': bq.LinearScale()} 

line = bq.Lines(y=y[0],x=x,scales=scales,colors=['red','green','cyan']) 

marks = [line]    # or marks = [line, scatt, hist, etc]


# Define axis, labels, attributes of axes
xax = bq.Axis(scale=scales['x'],
              label = 'seconds',
              grid_lines='solid')

yax = bq.Axis(scale=scales['y'],
              orientation='vertical',
              tickformat='0.2f',
              label = "g's (9.81 m/s^2)",
              grid_lines='solid')



### Using some help from `bqplot.interacts`  and `ipywidgets`



We are going to select the start and end times for each activity we seek to classify

 1. Walking
 
 2. Running 
 
 3. Idle 

For that we will create a **selector** that will help us to get the start and end times
and perform more analyses on the different sections of the data. We will also use a `PanZoom`
object that will be useful for helping us zoom, pan, and view the data better.<br/><br/>

We will create a `SelectMultiple` box to choose which variables to plot.

Lastly, we need to display/link the interval outputs to `HTML()` objects and such! For science!

In [89]:
from collections import OrderedDict
from bqplot.interacts import MultiSelector, PanZoom
from ipywidgets import ToggleButtons, SelectMultiple, VBox, HBox, HTML, interact

# link ipywidgets and bq interacts
from traitlets import link

columns = ['accelerometer_x', 'accelerometer_y', 'accelerometer_z',
           'gyroscope_x', 'gyroscope_y', 'gyroscope_z']

data = [df[col] for col in columns]

# Note that ipywidgets > 8.0 no longer uses odict for options, just a list
# bqplot depends on 7.6 at the moment

options = OrderedDict(zip(columns,data))
sel_options = list(iter(options.items()))
def change_line(change):
    line.y = [df[val] for val in change.new]

# SelectMultiple to choose which data to display

select_mult = SelectMultiple(options=columns,value=())
select_mult.observe(change_line, names='value')

# Selector callback, object and listener/observe function


def multi_callback(change):
    if st_end_interacts.brushing:
        intervals_html.value = str(st_end_interacts.selected)

    

st_end_interacts = MultiSelector(scale=scales['x'],marks=marks,names=["Walking","Running","Standing"])
st_end_interacts.observe(multi_callback, names=['selected'])


# scales for pan/zoom options
xsc = scales['x']
ysc = scales['y']
pz = PanZoom(scales={'x':[xsc], 'y': [ysc]})
pzx = PanZoom(scales={'x':[xsc]})
pzy = PanZoom(scales={'y': [ysc]})



# put interacts together into toggle-able button bar thing
interacts = ToggleButtons(options=OrderedDict([("PanZoomXY",pz),('MultiSelector',st_end_interacts),
                                              ("PanZoomX",pzx),("PanZoomY",pzy)]))
# Put HTML element to display the desired intervals for walking, running standing
intervals_html = HTML()

In [91]:
intervals_html.value = str([chr(10) + i for i in [str(s) for s in st_end_interacts.selected]])

fig1 = bq.Figure(marks=marks,
                 axes=[xax,yax],
                 animation_duration=1000,
                 title = "acc_data",)

fig1.layout.height = "400px"
fig1.layout.width = "500px"

link((interacts,'value'),(fig1,'interaction'))


VBox([HBox([select_mult,VBox([interacts,intervals_html])]),HBox([fig1])])


brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing
brushing


> <h1 style="color: red"> Caution </h1> </br>

It may look enticing to play with the data, but we have to acknowledge the data problem from before. The sensor does not lie perfectly on the foot. We want to define and implement our coordinates for the problem, since we don't want to have to recreate the model every time we put the sensor on.

![image.png](img/sensor_orientation.png) ![image.png](img/sensor_orientation2.png)

In [15]:
from scipy.fftpack import fft, fftfreq

xs = bq.LinearScale()
ys = bq.LinearScale()

xax = bq.Axis(scale=xs, label = 'frequency (hz)', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tickformat='0.2f', label = "magnitude/power spectrum", grid_lines='solid')


data = IMU_data.accelerometer_x.values[start:end]
time_step = 0.01

sample_freq = fftfreq(data.size, d = time_step)
y_fft = abs(fft(data))

# make the lines to be plotted
line2 = bq.Lines(x=sample_freq,y=y_fft, scales={'x':xs,'y':ys})



fig2 = bq.Figure(title='fft of acc',marks=[line2], axes=[xax,yax],animation_duration=1000)
plt.figure(fig=fig2)
plt.show()


In [ ]:
# This is magic: Watch the line and shape change as you
# swap 'accelerometer x' for 'accelerometer_y' or 'gyroscope_x'

line2.y = abs(fft(IMU_data.gyroscope_x.values[start:end]))
# set window/scale
ys.min = None
ys.max = 1000000
xs.min = -30
xs.max = 30

In [ ]:
peaks = [[sample_freq[i],yf[i]] for i in range(len(sample_freq)) if yf[i]>200 and sample_freq[i] > 0]
#print(peaks)
x = [peaks[i][0]  for i in range(len(peaks))]
y = [peaks[i][1]  for i in range(len(peaks))]

plt.scatter(x,y)
s

In [ ]:


import pandas as pd
import numpy as np

import sklearn
